### 見つかったRandomForestパラメータで予測をする
以下の組み合わせが最良だった
- n_estimators = 400
- max_depth=200
- min_samples_split = 2

#### 6/29 Preprocess_V4でモデルを作成しtrain_size=0.8で教師・検証データに分割した場合の検証データでのMAPEは10.01だった
#### ⇒このモデルでテストデータで予測して提出したところ、MAPEは12.39

#### 6/30 Preprocess_V5モデルで、検証したところMAPEは8.86

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [35]:
x_train = pd.read_csv('data/processed_train_goto_x.csv')
y_train = pd.read_csv('data/processed_train_goto_y.csv', header=None)
X_train, X_val , Y_train, Y_val = train_test_split( x_train, y_train, train_size=0.8)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [36]:
X_train = X_train.drop(['id','pj_no'],axis=1)

In [37]:
model = RandomForestRegressor(n_estimators=400, max_depth=200, min_samples_split=2,n_jobs=-1)
model.fit(X_train.values, Y_train.values.ravel() )

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=200,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [38]:
X_val = X_val.drop(['id','pj_no'],axis=1)

In [39]:
Y_pred = model.predict(X_val)

In [40]:
Y_pred

array([ 310615.00657711,  220387.14022532,  162276.91110985, ...,
        227976.4177957 ,  238656.66815884,  243491.99218797])

In [41]:
def mean_absolute_percentage_error( y_train: np.array, y_pred: np.array):
    diff = 0
    n = len(y_train)
    for i in range(n):
        diff += abs(y_train[i]-y_pred[i])/y_train[i]
    score = 100*diff / n
    
    return score

In [42]:
print( mean_absolute_percentage_error(Y_val.values, Y_pred))

[ 8.86310115]


In [43]:
from sklearn.externals import joblib
joblib.dump(model,'data/randomforest_v5.model')

['data/randomforest_v5.model']

In [44]:
importance = pd.DataFrame(model.feature_importances_, index=X_train.columns)
importance.to_csv("data/importance_submit2.csv")

## 以下はワーク用領域

In [16]:
X_train['tt_mseki'].head()

4434    105.99
4541     96.00
4391    105.16
443      94.00
4208     98.74
Name: tt_mseki, dtype: float64